# Phase 4: Quantum Noise and Decoherence

**Why Quantum Computers Are So Hard to Build**

---

## Table of Contents

1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Density Matrix Formalism](#density-matrices)
4. [Quantum Noise Channels](#noise-channels)
5. [T₁ and T₂ Decoherence](#decoherence)
6. [Real Quantum Hardware](#hardware)
7. [Why Error Correction is Needed](#error-correction)
8. [Summary & Key Takeaways](#summary)

---

**Author:** Wadoud Charbak  
**Based on:** Imperial College Quantum Information Theory - Section 4.1-4.2  
**For:** Quantinuum & Riverlane recruitment

---

## 1. Introduction <a id="introduction"></a>

This notebook explores the fundamental challenges facing quantum computing: **quantum noise** and **decoherence**.

### What We'll Learn

**Three Core Topics:**

1. **Density Matrix Formalism** - Describing mixed quantum states
2. **Quantum Noise Channels** - How qubits lose information
3. **T₁/T₂ Decoherence** - Time-dependent quantum decay

### Why This Matters

Phase 3 showed quantum algorithms with exponential speedup. But there's a catch:

- **Superconducting qubits:** T₂ ~ 50 μs → ~2,500 gates before decoherence
- **Error rates:** 0.1-1% per gate → errors accumulate rapidly
- **Complex algorithms:** Need thousands of gates
- **Result:** Quantum advantage destroyed by noise!

**This motivates Phase 5: Quantum Error Correction** 🛡️

### Key Questions

- Why do quantum computers need millikelvin temperatures?
- What is T₁ vs T₂ decoherence?
- How fast do quantum states decay?
- When does quantum advantage disappear?
- Why is error correction mandatory?

In [ ]:
# Setup paths
import sys
from pathlib import Path
notebook_dir = Path().absolute()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

# Core imports
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex
import warnings
warnings.filterwarnings('ignore')

# Phase 4 modules
from src.phase4_noise import (
    # Density matrices
    DensityMatrix,
    pure_state_density_matrix,
    mixed_state_density_matrix,
    fidelity,
    purity,
    bloch_vector,
    # Noise channels
    bit_flip_channel,
    phase_flip_channel,
    depolarizing_channel,
    amplitude_damping_channel,
    phase_damping_channel,
    # Decoherence
    DecoherenceSimulator,
    simulate_t1_decay,
    simulate_t2_decay,
    simulate_combined_decay,
    ramsey_experiment,
)

# Configure plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("✓ Setup complete!")
print("All Phase 4 modules loaded successfully")

---

## 2. Density Matrix Formalism <a id="density-matrices"></a>

### 2.1 Theory

**Pure states** are insufficient for real quantum systems that interact with their environment.

#### Definition

The **density matrix** (or density operator) is:

$$\rho = \sum_i p_i |\psi_i\rangle\langle\psi_i|$$

where $p_i$ are probabilities ($\sum_i p_i = 1$) and $|\psi_i\rangle$ are quantum states.

**For pure states:** $\rho = |\psi\rangle\langle\psi|$

#### Properties

1. **Hermitian:** $\rho = \rho^\dagger$
2. **Unit trace:** $\text{Tr}(\rho) = 1$
3. **Positive semi-definite:** $\langle\phi|\rho|\phi\rangle \geq 0$ for all $|\phi\rangle$

#### Purity

$$\text{Purity} = \text{Tr}(\rho^2) \in [1/d, 1]$$

- **Pure state:** $\text{Tr}(\rho^2) = 1$
- **Maximally mixed:** $\text{Tr}(\rho^2) = 1/d$ (where $d$ is dimension)

#### Von Neumann Entropy

$$S(\rho) = -\text{Tr}(\rho \log_2 \rho) = -\sum_i \lambda_i \log_2 \lambda_i$$

where $\lambda_i$ are eigenvalues of $\rho$.

- Pure state: $S(\rho) = 0$ bits
- Maximally mixed: $S(\rho) = \log_2 d$ bits

#### Bloch Vector (Single Qubit)

$$\rho = \frac{1}{2}(I + \vec{r} \cdot \vec{\sigma})$$

where $\vec{r} = (r_x, r_y, r_z)$ with $|\vec{r}| \leq 1$:

- **Pure states:** $|\vec{r}| = 1$ (on Bloch sphere surface)
- **Mixed states:** $|\vec{r}| < 1$ (inside Bloch sphere)
- **Maximally mixed:** $\vec{r} = 0$ (center)

In [ ]:
print("DENSITY MATRIX FORMALISM")
print("="*70)

# Create pure states
rho_0 = pure_state_density_matrix([1, 0])  # |0⟩
rho_1 = pure_state_density_matrix([0, 1])  # |1⟩
rho_plus = pure_state_density_matrix([1, 1])  # |+⟩
rho_minus = pure_state_density_matrix([1, -1])  # |−⟩

print("\n1. Pure States")
print("-"*70)
print("\nPure State |0⟩:")
print(rho_0)
print(f"\nMatrix:\n{rho_0.matrix}")
print(f"Bloch vector: {rho_0.bloch_vector()}")
print(f"On Bloch sphere: z-axis (north pole)")

In [ ]:
# Visualization: Bloch Sphere Representation
print("\n3. Bloch Sphere Visualization")
print("-"*70)

fig = plt.figure(figsize=(15, 5))

# Pure states
pure_states = {
    '|0⟩': rho_0,
    '|1⟩': rho_1,
    '|+⟩': rho_plus,
    '|−⟩': rho_minus
}

# Plot 1: Pure states on Bloch sphere
ax1 = fig.add_subplot(131, projection='3d')
for name, rho in pure_states.items():
    x, y, z = rho.bloch_vector()
    ax1.scatter([x], [y], [z], s=200, label=name, alpha=0.8)
    ax1.text(x*1.2, y*1.2, z*1.2, name, fontsize=12)

# Draw sphere
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
x_sphere = np.cos(u) * np.sin(v)
y_sphere = np.sin(u) * np.sin(v)
z_sphere = np.cos(v)
ax1.plot_wireframe(x_sphere, y_sphere, z_sphere, alpha=0.1, color='gray')

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Pure States on Bloch Sphere', fontsize=14, fontweight='bold')
ax1.legend()

# Plot 2: Purity comparison
ax2 = fig.add_subplot(132)
all_states = {'Pure |0⟩': rho_0, 'Pure |+⟩': rho_plus, '50% mixed': rho_mixed, 'Max mixed': rho_max_mixed}
names = list(all_states.keys())
purities = [rho.purity() for rho in all_states.values()]
colors = ['green', 'green', 'orange', 'red']

ax2.bar(names, purities, color=colors, alpha=0.7, edgecolor='black')
ax2.axhline(y=1.0, color='gray', linestyle='--', label='Pure (Tr(ρ²)=1)')
ax2.axhline(y=0.5, color='gray', linestyle=':', label='Max Mixed (Tr(ρ²)=0.5)')
ax2.set_ylabel('Purity', fontsize=12)
ax2.set_title('Purity Comparison', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Plot 3: Entropy comparison
ax3 = fig.add_subplot(133)
entropies = [rho.von_neumann_entropy() for rho in all_states.values()]
colors_ent = ['green', 'green', 'orange', 'red']

ax3.bar(names, entropies, color=colors_ent, alpha=0.7, edgecolor='black')
ax3.axhline(y=0, color='gray', linestyle='--', label='Pure (S=0)')
ax3.axhline(y=1.0, color='gray', linestyle=':', label='Max Mixed (S=1 bit)')
ax3.set_ylabel('Entropy (bits)', fontsize=12)
ax3.set_title('Von Neumann Entropy', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)
plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

print("\n✓ Pure states lie on Bloch sphere surface (|r| = 1)")
print("✓ Mixed states lie inside sphere (|r| < 1)")
print("✓ Purity = 1 for pure, < 1 for mixed")
print("✓ Entropy = 0 for pure, > 0 for mixed")

---

## 3. Quantum Noise Channels <a id="noise-channels"></a>

### 3.1 Theory: Kraus Operators

A **quantum channel** maps density matrices to density matrices:

$$\mathcal{E}(\rho) = \sum_i K_i \rho K_i^\dagger$$

where $\{K_i\}$ are **Kraus operators** satisfying:

$$\sum_i K_i^\dagger K_i = I$$

This ensures trace preservation: $\text{Tr}(\mathcal{E}(\rho)) = 1$

### 3.2 Six Fundamental Channels

#### 1. Bit-Flip Channel
**Physical:** $|0\rangle \leftrightarrow |1\rangle$ with probability $p$

**Kraus:** $K_0 = \sqrt{1-p} I$, $K_1 = \sqrt{p} \sigma_x$

#### 2. Phase-Flip Channel
**Physical:** $|+\rangle \leftrightarrow |-\rangle$ with probability $p$

**Kraus:** $K_0 = \sqrt{1-p} I$, $K_1 = \sqrt{p} \sigma_z$

#### 3. Depolarizing Channel
**Physical:** Random Pauli error

**Result:** $\rho \to (1-p)\rho + p I/2$

#### 4. Amplitude Damping (T₁)
**Physical:** $|1\rangle \to |0\rangle$ energy relaxation

**Time:** $\gamma(t) = 1 - e^{-t/T_1}$

#### 5. Phase Damping (T₂)
**Physical:** Coherence loss without energy loss

**Time:** $\lambda(t) = 1 - e^{-t/T_2}$

**Constraint:** $T_2 \leq 2T_1$ (always)

In [ ]:
print("\nQUANTUM NOISE CHANNELS")
print("="*70)

print("\n1. Bit-Flip Channel")
print("-"*70)
rho_initial = pure_state_density_matrix([1, 0])  # |0⟩

print(f"{'p (error)':<12} {'Purity':<12} {'P(|0⟩)':<12} {'P(|1⟩)'}")
print("-" * 50)
for p in [0.0, 0.1, 0.2, 0.5, 1.0]:
    rho_noisy = bit_flip_channel(rho_initial, p)
    prob_0 = rho_noisy.matrix[0, 0].real
    prob_1 = rho_noisy.matrix[1, 1].real
    print(f"{p:<12.2f} {rho_noisy.purity():<12.6f} {prob_0:<12.6f} {prob_1:<12.6f}")

print("\n→ As p increases, |0⟩ flips to |1⟩")

In [ ]:
# Visualization: Noise Channel Effects
print("\n3. Noise Channel Visualization")
print("-"*70)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Bit-flip effect on populations
ax1 = axes[0, 0]
error_probs = np.linspace(0, 1, 21)
rho_init = pure_state_density_matrix([1, 0])
pop_0 = []
pop_1 = []

for p in error_probs:
    rho_noisy = bit_flip_channel(rho_init, p)
    pop_0.append(rho_noisy.matrix[0, 0].real)
    pop_1.append(rho_noisy.matrix[1, 1].real)

ax1.plot(error_probs, pop_0, 'b-', linewidth=2, label='P(|0⟩)', marker='o')
ax1.plot(error_probs, pop_1, 'r-', linewidth=2, label='P(|1⟩)', marker='s')
ax1.set_xlabel('Error Probability p', fontsize=12)
ax1.set_ylabel('Population', fontsize=12)
ax1.set_title('Bit-Flip Channel: Population Transfer', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# 2. Depolarizing effect on purity
ax2 = axes[0, 1]
p_values = np.linspace(0, 1, 21)
purities = []
entropies = []

for p in p_values:
    rho_noisy = depolarizing_channel(rho_init, p)
    purities.append(rho_noisy.purity())
    entropies.append(rho_noisy.von_neumann_entropy())

ax2_twin = ax2.twinx()
line1 = ax2.plot(p_values, purities, 'g-', linewidth=2.5, label='Purity', marker='o')
line2 = ax2_twin.plot(p_values, entropies, 'orange', linewidth=2.5, label='Entropy', marker='s', linestyle='--')

ax2.set_xlabel('Depolarizing Parameter p', fontsize=12)
ax2.set_ylabel('Purity', fontsize=12, color='g')
ax2_twin.set_ylabel('Entropy (bits)', fontsize=12, color='orange')
ax2.set_title('Depolarizing Channel: Purity & Entropy', fontsize=13, fontweight='bold')
ax2.tick_params(axis='y', labelcolor='g')
ax2_twin.tick_params(axis='y', labelcolor='orange')
ax2.grid(alpha=0.3)

lines = line1 + line2
labels = [l.get_label() for l in lines]
ax2.legend(lines, labels, fontsize=11)

# 3. Phase-flip effect on coherence
ax3 = axes[1, 0]
rho_superpos = pure_state_density_matrix([1, 1])
coherences = []
bloch_x = []

for p in error_probs:
    rho_noisy = phase_flip_channel(rho_superpos, p)
    coherences.append(np.abs(rho_noisy.matrix[0, 1]))
    x, _, _ = rho_noisy.bloch_vector()
    bloch_x.append(x)

ax3.plot(error_probs, coherences, 'purple', linewidth=2.5, label='|ρ₀₁| (Coherence)', marker='o')
ax3.plot(error_probs, bloch_x, 'cyan', linewidth=2.5, label='Bloch x-coordinate', marker='s', linestyle='--')
ax3.set_xlabel('Error Probability p', fontsize=12)
ax3.set_ylabel('Magnitude', fontsize=12)
ax3.set_title('Phase-Flip Channel: Coherence Loss', fontsize=13, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(alpha=0.3)

# 4. Amplitude damping |1⟩ → |0⟩
ax4 = axes[1, 1]
rho_excited = pure_state_density_matrix([0, 1])
gamma_values = np.linspace(0, 1, 21)
pop_0_ad = []
pop_1_ad = []

for gamma in gamma_values:
    rho_damped = amplitude_damping_channel(rho_excited, gamma)
    pop_0_ad.append(rho_damped.matrix[0, 0].real)
    pop_1_ad.append(rho_damped.matrix[1, 1].real)

ax4.plot(gamma_values, pop_1_ad, 'r-', linewidth=2.5, label='P(|1⟩) = e⁻ᵞ', marker='o')
ax4.plot(gamma_values, pop_0_ad, 'b-', linewidth=2.5, label='P(|0⟩) = 1 - e⁻ᵞ', marker='s')
ax4.set_xlabel('Damping Parameter γ', fontsize=12)
ax4.set_ylabel('Population', fontsize=12)
ax4.set_title('Amplitude Damping (T₁): Energy Relaxation', fontsize=13, fontweight='bold')
ax4.legend(fontsize=11)
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Bit-flip: Transfers population between |0⟩ and |1⟩")
print("✓ Depolarizing: Decreases purity, increases entropy (mixing)")
print("✓ Phase-flip: Destroys coherence without population change")
print("✓ Amplitude damping: Energy relaxation |1⟩ → |0⟩")

---

## 4. T₁ and T₂ Decoherence <a id="decoherence"></a>

### 4.1 Theory

#### T₁: Energy Relaxation Time
- **Process:** $|1\rangle \to |0\rangle$ (excited → ground)
- **Physical:** Photon emission, energy loss to environment
- **Decay:** $P_1(t) = P_1(0) e^{-t/T_1}$
- **Affects:** Diagonal elements (populations)

#### T₂: Dephasing Time
- **Process:** Loss of quantum coherence
- **Physical:** Random phase accumulation
- **Decay:** $\rho_{01}(t) = \rho_{01}(0) e^{-t/T_2}$
- **Affects:** Off-diagonal elements (coherences)

#### Physical Constraint
$$T_2 \leq 2T_1$$

Energy relaxation causes dephasing, but pure dephasing can occur without energy loss.

#### Combined Evolution
Real systems: $\gamma(t) = 1 - e^{-t/T_1}$, $\lambda(t) = 1 - e^{-t/T_2}$

In [ ]:
print("\nT₁ AND T₂ DECOHERENCE")
print("="*70)

print("\n1. T₁ Decay: |1⟩ → |0⟩")
print("-"*70)
rho_excited = pure_state_density_matrix([0, 1])  # |1⟩
T1 = 100
times = np.linspace(0, 300, 50)

times_t1, states_t1 = simulate_t1_decay(rho_excited, T1, times)
pop_1 = [state.matrix[1, 1].real for state in states_t1]

print(f"T₁ = {T1}")
print(f"Initial P(|1⟩): {pop_1[0]:.4f}")
print(f"After t=T₁: {pop_1[len(pop_1)//3]:.4f} (expect ~0.368)")
print(f"After t=3T₁: {pop_1[-1]:.4f} (expect ~0.050)")
print("\n→ Energy dissipates to environment")
print("→ Population transfers from |1⟩ to |0⟩")

In [ ]:
# Visualization: Bloch Sphere Trajectory Under Decoherence
print("\n5. Bloch Sphere Trajectory: Watching Decoherence")
print("-"*70)

fig = plt.figure(figsize=(16, 5))

# Prepare different initial states
states_to_track = {
    '|+⟩': pure_state_density_matrix([1, 1]),
    '|1⟩': pure_state_density_matrix([0, 1]),
    '|i⟩': pure_state_density_matrix([1, 1j]),
}

T1, T2 = 100, 50
times = np.linspace(0, 200, 50)

# Three subplots for three different states
for idx, (name, rho_init) in enumerate(states_to_track.items()):
    ax = fig.add_subplot(1, 3, idx+1, projection='3d')
    
    # Simulate decoherence
    sim = DecoherenceSimulator(T1=T1, T2=T2, initial_state=rho_init)
    _, states = sim.simulate(times)
    
    # Extract Bloch vectors
    bloch_trajectory = [state.bloch_vector() for state in states]
    xs = [b[0] for b in bloch_trajectory]
    ys = [b[1] for b in bloch_trajectory]
    zs = [b[2] for b in bloch_trajectory]
    
    # Draw Bloch sphere
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x_sphere = np.cos(u) * np.sin(v)
    y_sphere = np.sin(u) * np.sin(v)
    z_sphere = np.cos(v)
    ax.plot_wireframe(x_sphere, y_sphere, z_sphere, alpha=0.1, color='gray')
    
    # Draw trajectory with color gradient
    colors_grad = plt.cm.coolwarm(np.linspace(0, 1, len(xs)))
    for i in range(len(xs) - 1):
        ax.plot(xs[i:i+2], ys[i:i+2], zs[i:i+2], color=colors_grad[i], linewidth=2, alpha=0.8)
    
    # Mark initial and final states
    ax.scatter([xs[0]], [ys[0]], [zs[0]], color='green', s=200, marker='o', 
              edgecolors='black', linewidth=2, label='Initial', zorder=5)
    ax.scatter([xs[-1]], [ys[-1]], [zs[-1]], color='red', s=200, marker='X', 
              edgecolors='black', linewidth=2, label='Final', zorder=5)
    
    # Draw axes
    ax.plot([0, 1.3], [0, 0], [0, 0], 'k-', alpha=0.3, linewidth=1)
    ax.plot([0, 0], [0, 1.3], [0, 0], 'k-', alpha=0.3, linewidth=1)
    ax.plot([0, 0], [0, 0], [0, 1.3], 'k-', alpha=0.3, linewidth=1)
    ax.text(1.4, 0, 0, 'X', fontsize=12)
    ax.text(0, 1.4, 0, 'Y', fontsize=12)
    ax.text(0, 0, 1.4, 'Z', fontsize=12)
    
    ax.set_xlabel('X', fontsize=10)
    ax.set_ylabel('Y', fontsize=10)
    ax.set_zlabel('Z', fontsize=10)
    ax.set_title(f'State {name} Decoherence\nT₁={T1}, T₂={T2}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9, loc='upper right')
    
    # Set viewing angle
    ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

print("\n✓ All states spiral inward toward center of Bloch sphere")
print("✓ Pure states (surface) → Mixed states (interior)")
print("✓ T₂ dephasing causes radial decay")
print("✓ T₁ relaxation pulls states toward |0⟩ (north pole)")
print("✓ Final state closer to maximally mixed (center)")

In [ ]:
print("\n2. T₂ Dephasing: |+⟩ Losing Coherence")
print("-"*70)
rho_superpos = pure_state_density_matrix([1, 1])  # |+⟩
T2 = 50
times = np.linspace(0, 150, 50)

times_t2, states_t2 = simulate_t2_decay(rho_superpos, T2, times)
coherence = [np.abs(state.matrix[0, 1]) for state in states_t2]
pop_0 = [state.matrix[0, 0].real for state in states_t2]

print(f"T₂ = {T2}")
print(f"Initial coherence: {coherence[0]:.4f}")
print(f"After t=3T₂: {coherence[-1]:.4f}")
print(f"Coherence remaining: {coherence[-1]/coherence[0]*100:.1f}%")
print(f"\nPopulations unchanged: P(|0⟩) = {pop_0[0]:.4f} → {pop_0[-1]:.4f}")
print("\n→ Off-diagonal elements decay")
print("→ No energy loss - pure dephasing")

In [ ]:
# Visualization: T₁ and T₂ Decoherence Over Time
print("\n4. T₁/T₂ Decoherence Visualization")
print("-"*70)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Prepare data
rho_excited = pure_state_density_matrix([0, 1])
rho_superpos = pure_state_density_matrix([1, 1])
T1, T2 = 100, 50
times = np.linspace(0, 300, 100)

# T1 decay
times_t1, states_t1 = simulate_t1_decay(rho_excited, T1, times)
pop_1_t1 = [state.matrix[1, 1].real for state in states_t1]
pop_0_t1 = [state.matrix[0, 0].real for state in states_t1]

# T2 decay
times_t2, states_t2 = simulate_t2_decay(rho_superpos, T2, times)
coherence_t2 = [np.abs(state.matrix[0, 1]) for state in states_t2]

# Combined T1+T2
sim = DecoherenceSimulator(T1=T1, T2=T2, initial_state=rho_superpos)
_, fids = sim.compute_fidelity_decay(times)
_, purs = sim.compute_purity_decay(times)

# Plot 1: T₁ Population Evolution
ax1 = axes[0, 0]
ax1.plot(times, pop_1_t1, 'r-', linewidth=2.5, label='P(|1⟩) = exp(-t/T₁)')
ax1.plot(times, pop_0_t1, 'b-', linewidth=2.5, label='P(|0⟩) = 1 - exp(-t/T₁)')
ax1.axvline(T1, color='gray', linestyle='--', alpha=0.5, label=f'T₁ = {T1}')
ax1.axhline(np.exp(-1), color='red', linestyle=':', alpha=0.5)
ax1.text(T1 + 5, np.exp(-1) + 0.05, '≈0.368', fontsize=10)
ax1.set_xlabel('Time', fontsize=12)
ax1.set_ylabel('Population', fontsize=12)
ax1.set_title('T₁ Decay: |1⟩ → |0⟩ Energy Relaxation', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: T₂ Coherence Evolution
ax2 = axes[0, 1]
ax2.plot(times, coherence_t2, 'purple', linewidth=2.5, label='|ρ₀₁(t)| = 0.5·exp(-t/T₂)')
ax2.axvline(T2, color='gray', linestyle='--', alpha=0.5, label=f'T₂ = {T2}')
ax2.axhline(0.5*np.exp(-1), color='purple', linestyle=':', alpha=0.5)
ax2.text(T2 + 5, 0.5*np.exp(-1) + 0.02, '≈0.184', fontsize=10)
ax2.set_xlabel('Time', fontsize=12)
ax2.set_ylabel('Coherence |ρ₀₁|', fontsize=12)
ax2.set_title('T₂ Dephasing: Coherence Loss (No Energy Loss)', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: Fidelity Decay
ax3 = axes[1, 0]
ax3.plot(times, fids, 'green', linewidth=2.5, label='F(ρ(t), ρ(0))')
ax3.axvline(T2, color='gray', linestyle='--', alpha=0.5, label=f'T₂ = {T2}')
ax3.axvline(T1, color='gray', linestyle=':', alpha=0.5, label=f'T₁ = {T1}')
ax3.axhline(0.5, color='orange', linestyle='--', alpha=0.5)
ax3.text(5, 0.53, 'Orthogonal (F=0.5)', fontsize=10)
ax3.set_xlabel('Time', fontsize=12)
ax3.set_ylabel('Fidelity', fontsize=12)
ax3.set_title('Combined T₁+T₂: Fidelity Decay', fontsize=13, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(alpha=0.3)

# Plot 4: Purity Decay
ax4 = axes[1, 1]
ax4.plot(times, purs, 'blue', linewidth=2.5, label='Tr(ρ²(t))')
ax4.axvline(T2, color='gray', linestyle='--', alpha=0.5, label=f'T₂ = {T2}')
ax4.axvline(T1, color='gray', linestyle=':', alpha=0.5, label=f'T₁ = {T1}')
ax4.axhline(1.0, color='green', linestyle='--', alpha=0.3)
ax4.axhline(0.5, color='red', linestyle='--', alpha=0.3)
ax4.text(5, 1.02, 'Pure', fontsize=10)
ax4.text(5, 0.52, 'Maximally Mixed', fontsize=10)
ax4.set_xlabel('Time', fontsize=12)
ax4.set_ylabel('Purity Tr(ρ²)', fontsize=12)
ax4.set_title('Combined T₁+T₂: Purity Decay', fontsize=13, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ T₁: Population relaxes exponentially (|1⟩ → |0⟩)")
print("✓ T₂: Coherence decays exponentially (superposition lost)")
print("✓ T₂ < 2T₁: Dephasing faster than energy relaxation")
print("✓ Fidelity: State becomes increasingly different from initial")
print("✓ Purity: State becomes mixed (approaches 0.5)")

---

## 5. Real Quantum Hardware <a id="hardware"></a>

### 5.1 Hardware Platforms

| Platform | T₁ | T₂ | Temperature | Gate Time | Gates/T₂ |
|----------|----|----|-------------|-----------|----------|
| **Superconducting** | 50-100 μs | 20-80 μs | 10-20 mK | 20-50 ns | ~2,000 |
| **Ion Traps** | >10 s | ~1 s | Room temp | 10-100 μs | ~20,000 |
| **NV Centers** | ~1 ms | ~1 ms | Room temp | ~100 ns | ~10,000 |

### 5.2 Why Different Coherence?

**Superconducting:**
- Fast gates (~20 ns)
- Requires extreme cooling (mK)
- Sensitive to flux noise → T₂ < T₁

**Ion Traps:**
- Excellent coherence (seconds!)
- Well isolated from environment
- But slow gates (~10-100 μs)

In [ ]:
# Visualization: Hardware Platform Comparison
print("\n2. Hardware Platform Comparison")
print("-"*70)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Hardware parameters (normalized for visualization)
platforms = ['Superconducting\n(IBM/Google)', 'Ion Trap\n(IonQ)', 'NV Center\n(Diamond)']
T1_times = [100e-6, 10, 1e-3]  # seconds
T2_times = [50e-6, 1, 1e-3]  # seconds
gate_times = [20e-9, 50e-6, 100e-9]  # seconds
temps = [0.02, 300, 300]  # Kelvin

# Convert to microseconds for display
T1_us = [t * 1e6 for t in T1_times]
T2_us = [t * 1e6 for t in T2_times]
gate_us = [t * 1e6 for t in gate_times]

# Plot 1: Coherence Times
ax1 = axes[0]
x_pos = np.arange(len(platforms))
width = 0.35

bars1 = ax1.bar(x_pos - width/2, T1_us, width, label='T₁', color='steelblue', alpha=0.8, edgecolor='black')
bars2 = ax1.bar(x_pos + width/2, T2_us, width, label='T₂', color='coral', alpha=0.8, edgecolor='black')

ax1.set_ylabel('Time (μs)', fontsize=12)
ax1.set_title('Coherence Times (T₁ and T₂)', fontsize=13, fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(platforms, fontsize=10)
ax1.legend(fontsize=11)
ax1.set_yscale('log')
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    if height > 1000:
        ax1.text(bar.get_x() + bar.get_width()/2., height * 1.5, f'{height/1e6:.1f}s', 
                ha='center', va='bottom', fontsize=9)
    else:
        ax1.text(bar.get_x() + bar.get_width()/2., height * 1.5, f'{height:.0f}μs', 
                ha='center', va='bottom', fontsize=9)

# Plot 2: Gates Before Decoherence
ax2 = axes[1]
gates_before_decay = [T2_times[i] / gate_times[i] for i in range(len(platforms))]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

bars = ax2.bar(platforms, gates_before_decay, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Number of Gates', fontsize=12)
ax2.set_title('Gates Before Decoherence (T₂/t_gate)', fontsize=13, fontweight='bold')
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3)

for i, bar in enumerate(bars):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height * 1.5, 
            f'{int(height):,}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 3: Operating Temperature
ax3 = axes[2]
colors_temp = ['#3498DB', '#E74C3C', '#E74C3C']
bars_temp = ax3.bar(platforms, temps, color=colors_temp, alpha=0.8, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('Temperature (K)', fontsize=12)
ax3.set_title('Operating Temperature', fontsize=13, fontweight='bold')
ax3.set_yscale('log')
ax3.grid(axis='y', alpha=0.3)

for i, bar in enumerate(bars_temp):
    height = bar.get_height()
    if height < 1:
        label = f'{height*1000:.0f} mK'
    else:
        label = f'{height:.0f} K\n(Room temp)'
    ax3.text(bar.get_x() + bar.get_width()/2., height * 1.5, 
            label, ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Superconducting: Fast gates, short coherence, extreme cooling")
print("✓ Ion traps: Excellent coherence, slower gates, room temperature")
print("✓ NV centers: Good coherence, room temperature, fast gates")
print("✓ Trade-off: Speed vs. coherence vs. operating conditions")

---

## 6. Why Error Correction is Needed <a id="error-correction"></a>

### 6.1 The Problem: Error Accumulation

Modern error rates:
- Single-qubit: ~0.1%
- Two-qubit: ~0.5-1%
- Measurement: ~1-2%

For $n$ gates with error rate $p$:
$$P(\text{success}) \approx (1 - p)^n$$

**Example with $p = 0.001$ (0.1%):**
- 100 gates: 90% success
- 1,000 gates: 37% success
- 10,000 gates: 0.005% success ❌

### 6.2 The Solution

**Quantum Error Correction (Phase 5):**
- Encode logical qubits into multiple physical qubits
- Detect errors without collapsing superposition
- Correct errors before they accumulate
- **Cost:** 100x-1000x qubit overhead!

In [ ]:
# Visualization: Error Accumulation and Circuit Depth
print("\nERROR ACCUMULATION ANALYSIS")
print("="*70)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Success probability vs number of gates
ax1 = axes[0]
n_gates = np.arange(1, 10001, 50)
error_rates = [0.0001, 0.001, 0.005, 0.01]
labels = ['0.01% (Fault-tolerant target)', '0.1% (Best today)', '0.5%', '1%']
colors = ['green', 'blue', 'orange', 'red']

for i, p in enumerate(error_rates):
    success_prob = (1 - p) ** n_gates
    ax1.plot(n_gates, success_prob, linewidth=2.5, label=labels[i], color=colors[i])

ax1.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax1.text(5000, 0.53, '50% success threshold', fontsize=10)
ax1.set_xlabel('Number of Gates', fontsize=12)
ax1.set_ylabel('Success Probability', fontsize=12)
ax1.set_title('Circuit Success vs Gate Count', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.set_yscale('log')
ax1.grid(alpha=0.3)
ax1.set_ylim([1e-5, 1])

# Plot 2: Maximum circuit depth for different success targets
ax2 = axes[1]
success_targets = [0.99, 0.95, 0.9, 0.5]
error_rates_fine = np.logspace(-4, -2, 50)  # 0.01% to 1%

for target in success_targets:
    max_gates = np.log(target) / np.log(1 - error_rates_fine)
    ax2.plot(error_rates_fine * 100, max_gates, linewidth=2.5, label=f'{target*100:.0f}% success')

# Mark current hardware
ax2.axvline(x=0.1, color='blue', linestyle='--', alpha=0.5, linewidth=2)
ax2.text(0.12, 5000, 'Best single-qubit\ntoday (~0.1%)', fontsize=9, color='blue')
ax2.axvline(x=0.5, color='orange', linestyle='--', alpha=0.5, linewidth=2)
ax2.text(0.52, 5000, 'Two-qubit\ngates (~0.5%)', fontsize=9, color='orange')
ax2.axvline(x=0.01, color='green', linestyle='--', alpha=0.5, linewidth=2)
ax2.text(0.012, 50000, 'QEC target\n(~0.01%)', fontsize=9, color='green')

ax2.set_xlabel('Gate Error Rate (%)', fontsize=12)
ax2.set_ylabel('Maximum Circuit Depth (gates)', fontsize=12)
ax2.set_title('Maximum Circuit Depth Before Failure', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(alpha=0.3)
ax2.set_xlim([0.01, 2])
ax2.set_ylim([10, 100000])

plt.tight_layout()
plt.show()

# Table of circuit depths
print("\nMaximum Circuit Depth (gates) for 90% Success Rate:\n")
print(f"{'Error Rate':<15} {'Max Gates':<15} {'Algorithm Feasibility'}")
print("-" * 60)
examples = [
    (0.0001, "Surface codes (fault-tolerant)", "✓ Shor's algorithm"),
    (0.001, "Best case scenario", "✓ VQE, QAOA"),
    (0.005, "Current two-qubit gates", "Limited depth"),
    (0.01, "NISQ era typical", "Very limited"),
]

for p, scenario, algo in examples:
    max_g = int(np.log(0.9) / np.log(1 - p))
    print(f"{p*100:>5.2f}%  {scenario:<30} {max_g:>10,}  {algo}")

print("\n→ Without QEC: Circuit depth severely limited by errors")
print("→ Complex algorithms need 1,000-100,000+ gates")
print("→ Quantum error correction mandatory for practical QC")

---

## 7. Summary & Key Takeaways <a id="summary"></a>

### What We Learned

#### 1. Density Matrix Formalism
- **Pure states:** $\text{Tr}(\rho^2) = 1$, entropy = 0
- **Mixed states:** $\text{Tr}(\rho^2) < 1$, entropy > 0
- **Bloch sphere:** Pure on surface, mixed inside
- **Fidelity:** Measures state similarity

#### 2. Quantum Noise
- **Kraus operators:** $\mathcal{E}(\rho) = \sum_i K_i \rho K_i^\dagger$
- **Six channels:** Bit-flip, phase-flip, depolarizing, amplitude/phase damping
- **CPTP maps:** Completely Positive, Trace Preserving

#### 3. Decoherence
- **T₁:** Energy relaxation ($|1\rangle \to |0\rangle$)
- **T₂:** Dephasing (coherence loss)
- **Constraint:** $T_2 \leq 2T_1$ always
- **Exponential decay:** $e^{-t/T}$

#### 4. Real Hardware
- **Superconducting:** T₂~50μs, ~2,500 gates
- **Ion traps:** T₂~1s, ~20,000 gates
- **Trade-offs:** Speed vs coherence

#### 5. Why Error Correction
- **Errors accumulate:** $(1-p)^n$ → exponential failure
- **Complex algorithms:** Need thousands of gates
- **Solution:** Quantum error correction
- **Cost:** 100x-1000x qubit overhead

### Key Insights

**Why quantum computers are hard:**
- Extreme isolation needed (mK temperatures)
- Decoherence limits circuit depth
- Errors accumulate rapidly
- Cannot copy quantum information

**This motivates Phase 5:**
- 3-qubit bit-flip code
- Shor's 9-qubit code
- Stabilizer formalism
- Fault-tolerant quantum computing

---

## 🎉 Phase 4 Complete!

You now understand:
✅ Density matrix formalism  
✅ Six fundamental noise channels  
✅ T₁/T₂ decoherence mechanisms  
✅ Real hardware limitations  
✅ Why error correction is mandatory  

**Ready for Phase 5: Quantum Error Correction!** 🛡️

---

**Author:** Wadoud Charbak  
**Based on:** Imperial College Quantum Information Theory  
**For:** Quantinuum & Riverlane recruitment